In [ ]:
import rtsvg
import string
import random
import panel as pn
import param
import threading
import numpy as np
pn.extension(design="material", sizing_mode="stretch_width")
rt = rtsvg.RACETrack()
def makeWord(n): return ''.join(random.choice(string.ascii_lowercase) for _ in range(n))
docs, m = [], 10
for i in range(m):
    doc_length = 100 + random.randint(0,1000)
    words = []
    for j in range(doc_length): words.append(makeWord(3+random.randint(0,5)))
    docs.append(' '.join(words))
tbs  = []
for doc in docs: tbs.append(rt.textBlock(doc, word_wrap=True))
prs  = []
_lu_ = {'[abc][def]': '#ff0000', '[ghi][jkl]': '#00ff00', '[mno][pqr]': '#0000ff'}
tbs = sorted(tbs)
for tb in tbs: prs.append(tb.pixelRepr(_lu_))

class TileSelector(param.Parameterized):
    def __init__(self, rt_self, tiles, per_row=5, *args, **kwargs):
        super().__init__(**kwargs)
        self.rt_self = rt_self
        self.tiles   = tiles
        self.per_row = per_row
        self._column_ = pn.Column(pn.pane.HTML(rt.tile(self.tiles, spacer=10)._repr_svg_()))
    def _update_panel(self):
        pass
    def panel(self):
        return self._column_
_selector_ = TileSelector(rt, prs)
_selector_.panel()

In [ ]:
pn.Column(pn.Row(pn.widgets.Button(name='Clr'), pn.widgets.Button(name='All'), 
                 pn.widgets.Button(name='Grp'), pn.widgets.Button(name='UnGrp'),
                 pn.widgets.Button(name='Del'),
                 pn.widgets.ToggleGroup(name='Reset', options=['s','+','-','^'], behavior='radio'),))

In [ ]:
from panel.reactive import ReactiveHTML
class RTTileSelector(ReactiveHTML):
    #
    # Panel Template
    # - The following is re-written in the constructor
    #
    _template = """
        <svg id="parent" x="0" y="0" width="400" height="300">
            <rect id="background" x="0" y="0" width="400" height="300" fill="#ffffff" />
            <svg id="tiles" width="400" height="300">
                <rect id="r0" x="10"  y="10" width="20" height="20" fill="#a0a0a0" stroke="#00ff00" stroke-width="2.0"/>
            </svg>
            <rect id="drag" x="-10" y="-10" width="5" height="5" fill="#ffffff" opacity="0.6" />
            <rect id="screen" x="0" y="0" width="400" height="300" opacity="0.05" 
              onmousedown="${script('myonmousedown')}"
              onmousemove="${script('myonmousemove')}"
              onmouseup="${script('myonmouseup')}"
            />
        </svg>
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.lock = threading.Lock()
        self.tile_geoms = {
            'r0':( 10,10,20,20),
            'r1':( 40,10,20,20),
            'r2':( 70,10,20,20),
            'r3':(100,10,20,20)
        }
        self.param.watch(self.applyDragOp,   'drag_op_finished')
    
    x0_middle          = param.Integer(default=0)
    y0_middle          = param.Integer(default=0)
    x1_middle          = param.Integer(default=0)
    y1_middle          = param.Integer(default=0)

    drag_op_finished = param.Boolean(default=False)
    drag_x0          = param.Integer(default=0)
    drag_y0          = param.Integer(default=0)
    drag_x1          = param.Integer(default=10)
    drag_y1          = param.Integer(default=10)
    drag_shiftkey    = param.Boolean(default=False)

    async def applyDragOp(self,event):
        self.lock.acquire()
        try:
            print(event)
            if self.drag_op_finished:
                _x0,_y0,_x1,_y1 = self.drag_x0, self.drag_y0, self.drag_x1, self.drag_y1
                if _x0 == _x1: _x1 += 1
                if _y0 == _y1: _y1 += 1
                # Mark operation as finished
                self.drag_op_finished = False
        finally:
            self.lock.release()

    #
    # Panel Javascript Definitions
    #
    _scripts = {
        'render':"""
            state.x0_drag  = state.y0_drag = -10;
            state.x1_drag  = state.y1_drag =  -5;
            state.shiftkey = false;
            state.drag_op  = false;
        """,
        'myonmousemove':"""
            event.preventDefault();
            if (state.drag_op) {
                state.x1_drag  = event.offsetX;
                state.y1_drag  = event.offsetY;
                state.shiftkey = event.shiftKey;
                self.myUpdateDragRect();
            }
        """,
        'myonmousedown':"""
            event.preventDefault();
            if (event.button == 0) {
                state.x0_drag  = event.offsetX;
                state.y0_drag  = event.offsetY;
                state.x1_drag  = event.offsetX+1;
                state.y1_drag  = event.offsetY+1;
                state.drag_op  = true;
                state.shiftkey = event.shiftKey;
                self.myUpdateDragRect();
            } else if (event.button == 1) {
                data.x0_middle = data.x1_middle = event.offsetX;
                data.y0_middle = data.y1_middle = event.offsetY;
            }
        """,
        'myonmouseup':"""
            event.preventDefault();
            if (state.drag_op && event.button == 0) {
                state.x1_drag  = event.offsetX;
                state.y1_drag  = event.offsetY;
                state.shiftkey = event.shiftKey;
                state.drag_op  = false;
                self.myUpdateDragRect();
                data.drag_x0          = state.x0_drag;
                data.drag_y0          = state.y0_drag;
                data.drag_x1          = state.x1_drag;
                data.drag_y1          = state.y1_drag;
                data.drag_shiftkey    = state.shiftkey
                data.drag_op_finished = true;
            } else if (event.button == 1) {
                data.x1_middle          = event.offsetX;
                data.y1_middle          = event.offsetY;
            }
        """,
        'myUpdateDragRect':"""
            if (state.drag_op) {
                x = state.x0_drag; 
                if (state.x1_drag < x) { x = state.x1_drag; }
                y = state.y0_drag; 
                if (state.y1_drag < y) { y = state.y1_drag; }
                w = Math.abs(state.x1_drag - state.x0_drag)
                h = Math.abs(state.y1_drag - state.y0_drag)
                drag.setAttribute('x',x);     drag.setAttribute('y',y);
                drag.setAttribute('width',w); drag.setAttribute('height',h);
                if (state.shiftkey) { drag.setAttribute('stroke','#ff0000'); }
                else                { drag.setAttribute('stroke','#000000'); }
            } else {
                drag.setAttribute('x',-10);   drag.setAttribute('y',-10);
                drag.setAttribute('width',5); drag.setAttribute('height',5);
            }
        """}

RTTileSelector()